In [3]:
%cd ..

c:\Users\Daniel\cyanobacteria-monitoring


In [191]:
import numpy as np

import pandas as pd

import plotly.express as px

from sklearn.compose import TransformedTargetRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.manifold import TSNE
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer

import yaml

from src.data import clean_gee

In [ ]:
with open("params.yaml") as config_file:
    config = yaml.safe_load(config_file)

In [350]:
pd.read_csv(config["data_load"]["labeled_df"])

,date,NDVI_median,NDCI_median,B1_median,B2_median,B3_median,B4_median,B5_median,B6_median,B7_median,B8_median,B8A_median,B9_median,B11_median,B12_median,Resultado,Data da coleta,interval
0,2019-01-28,-0.396403,-0.034596,0.0419,0.0560,0.0794,0.0870,0.0830,0.0431,0.04220,0.0378,0.0322,0.0120,0.0245,0.0212,251,2019-01-28,7
1,2019-01-30,-0.523918,-0.040864,0.0267,0.0418,0.0617,0.0671,0.0618,0.0243,0.02690,0.0209,0.0169,0.0406,0.0137,0.0132,251,2019-01-28,7
2,2019-02-07,-0.616129,-0.057269,0.0230,0.0302,0.0519,0.0484,0.0429,0.0138,0.01500,0.0116,0.0081,0.0035,0.0049,0.0044,335,2019-02-11,9
3,2019-02-09,-0.418552,-0.029911,0.0294,0.0441,0.0646,0.0631,0.0597,0.0263,0.02950,0.0259,0.0202,0.0419,0.0189,0.0165,335,2019-02-11,9
4,2019-02-22,-0.419355,-0.044293,0.0302,0.0467,0.0671,0.0579,0.0528,0.0250,0.02680,0.0236,0.0195,0.0090,0.0166,0.0143,298,2019-02-25,12
5,2019-03-14,-0.634973,-0.079942,0.0256,0.0419,0.0697,0.0744,0.0635,0.0206,0.02190,0.0166,0.0111,0.0016,0.0053,0.0042,76,2019-03-18,16
6,2019-03-16,-0.551167,-0.060256,0.0434,0.0501,0.0812,0.0874,0.0774,0.0314,0.03140,0.0253,0.0200,0.0142,0.0076,0.0065,76,2019-03-18,16
7,2019-03-24,-0.614657,-0.095912,0.0285,0.0432,0.0675,0.0682,0.0562,0.0199,0.02120,0.0162,0.0122,0.0012,0.0055,0.0042,186,2019-03-26,18
8,2019-03-26,-0.596131,-0.088013,0.0397,0.0449,0.0702,0.0661,0.0555,0.0199,0.01990,0.0167,0.0122,0.0036,0.0076,0.0066,186,2019-03-26,18
9,2019-04-10,-0.674556,-0.117098,0.0235,0.0386,0.0609,0.0555,0.0435,0.0130,0.01450,0.0109,0.0079,0.0007,0.0051,0.0044,26,2019-04-09,21


In [289]:
df_train = pd.read_csv(config["data_split"]["trainset_path"])

df_train = df_train.loc[~df_train["date"].isin(["2020-08-07",
    "2020-12-28",
    "2021-01-22",
    "2021-04-14",
    "2019-06-17",])]
    


# 2020-08-07: nuvem
# 2020-12-28: nuvem
# 2021-01-22: escuro
# 2021-04-14: duas datas
# 2019-06-17 -> sombra

In [290]:
poly = PolynomialFeatures(2, include_bias=False)

y_train = df_train["Resultado"]

df_bands = df_train[['NDVI_median', 'NDCI_median', 'B1_median', 'B2_median',
       'B3_median', 'B4_median', 'B5_median', 'B6_median', 'B7_median',
       'B8_median', 'B8A_median', 'B9_median', 'B11_median', 'B12_median',
        'B3_B2', 'B5_B2', 'B3_B4',
       'B5_B4', 'B5B3_B2']]

df_bands["delta_days"] = (pd.to_datetime(df_train["date"]) - pd.to_datetime(df_train["Data da coleta"])).dt.days

df_poly = poly.fit_transform(df_bands)

df_poly = pd.DataFrame(df_poly, columns=poly.get_feature_names(df_bands.columns))

<ipython-input-290-4c3cce41cb5e>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [291]:
mi = mutual_info_regression(df_poly, y_train)

mi_df = pd.DataFrame({
    "ft": df_poly.columns,
    "mi": mi
})

In [292]:
mi_df.sort_values("mi", ascending=False)

,ft,mi
119,B4_median B3_B2,0.210832
213,B3_B2 B5B3_B2,0.197277
194,B11_median^2,0.195287
190,B9_median B3_B4,0.194750
137,B5_median B5B3_B2,0.193454
...,...,...
80,B2_median B5_median,0.000000
77,B2_median^2,0.000000
76,B1_median delta_days,0.000000
74,B1_median B5_B4,0.000000


In [293]:
mi_df.loc[mi_df["mi"]>0.15, "ft"].values

array(['B6_median', 'B3_B4', 'NDVI_median B8_median',
       'NDCI_median B8_median', 'B1_median B4_median', 'B4_median B3_B2',
       'B5_median B5B3_B2', 'B6_median^2', 'B9_median B3_B2',
       'B9_median B3_B4', 'B11_median^2', 'B11_median B12_median',
       'B3_B2 B5_B4', 'B3_B2 B5B3_B2', 'B5_B2 B3_B4', 'B3_B4^2',
       'B3_B4 B5B3_B2', 'B5_B4 B5B3_B2'], dtype=object)

In [308]:
lasso = Lasso(max_iter=1e9, alpha=1)
lasso.fit(df_poly, y_train)

Lasso(alpha=1, max_iter=1000000000.0)

In [312]:
lasso_df = pd.DataFrame({"ft": df_poly.columns, "lasso": lasso.coef_})
fts = pd.merge(mi_df, lasso_df, on="ft")
fts.loc[fts["mi"] * fts["lasso"] != 0]

,ft,mi,lasso
16,B3_B4,0.154927,70234.858111
18,B5B3_B2,0.090404,-49790.632525
35,NDVI_median B5_B2,0.050277,-23536.379195
56,NDCI_median B5_B4,0.030964,-61701.635922
75,B1_median B5B3_B2,0.118343,-21346.926639
123,B4_median B5B3_B2,0.064754,-20765.184838
183,B8A_median B5B3_B2,0.027765,26747.521636
192,B9_median B5B3_B2,0.058107,17107.633867
222,B3_B4 B5B3_B2,0.164334,-4579.493797
224,B5_B4^2,0.054502,-6906.500353


In [330]:
fts["abs_lasso"] = abs(fts["lasso"])
px.scatter(fts, x="mi", y="abs_lasso", hover_data=["ft"])

In [340]:
fts.query("mi > 0.17")

,ft,mi,lasso,abs_lasso
119,B4_median B3_B2,0.210832,-0.0,0.0
137,B5_median B5B3_B2,0.193454,-0.0,0.0
139,B6_median^2,0.186901,0.0,0.0
190,B9_median B3_B4,0.194750,0.0,0.0
194,B11_median^2,0.195287,0.0,0.0
213,B3_B2 B5B3_B2,0.197277,0.0,0.0


In [347]:
fts.loc[(fts["mi"] > .17) | (fts["abs_lasso"] > 0) & (fts["mi"] > 0)].sort_values("ft")["ft"].values

array(['B11_median^2', 'B1_median B5B3_B2', 'B3_B2 B5B3_B2', 'B3_B4',
       'B3_B4 B5B3_B2', 'B4_median B3_B2', 'B4_median B5B3_B2', 'B5B3_B2',
       'B5B3_B2^2', 'B5_B4^2', 'B5_median B5B3_B2', 'B6_median^2',
       'B8A_median B5B3_B2', 'B9_median B3_B4', 'B9_median B5B3_B2',
       'NDCI_median B5_B4', 'NDVI_median B5_B2'], dtype=object)

In [296]:
#df_poly_selected = df_poly[mi_df.loc[mi_df["mi"]>0.10, "ft"].values]

cols = pd.DataFrame({"ft": df_poly.columns, "lasso": lasso.coef_}).query("lasso != 0").sort_values("lasso", ascending=False)["ft"].values
df_poly_selected = df_poly[cols]

In [297]:
linreg = LinearRegression()
model = TransformedTargetRegressor(
        regressor=linreg, transformer=PowerTransformer(method="yeo-johnson")
    )

model.fit(df_poly_selected, y_train)

y_pred = np.where(model.predict(df_poly_selected) < 0, 0, model.predict(df_poly_selected))
resid =  y_train - y_pred

In [300]:
df_poly_selected

,delta_days,B5B3_B2^2,B3_B4 delta_days,B3_B4 B5_B4,B5_B2 delta_days,B9_median delta_days,delta_days^2,NDVI_median B5_B2,B3_B2 B3_B4,B4_median B5B3_B2,B3_B2 delta_days,B5_B4 delta_days,B5B3_B2
0,0.0,8.410000,0.000000,0.870683,0.000000,0.0000,0.0,-0.587527,1.293998,0.252300,0.000000,0.000000,2.900000
1,2.0,8.729339,1.839046,0.846893,2.956938,0.0812,4.0,-0.774596,1.357286,0.198250,2.952153,1.842027,2.954545
2,-4.0,9.853778,-4.289256,0.950460,-5.682119,-0.0140,16.0,-0.875230,1.842818,0.151931,-6.874172,-3.545455,3.139073
3,-2.0,7.944473,-2.047544,0.968608,-2.707483,-0.0838,4.0,-0.566611,1.499675,0.177853,-2.929705,-1.892235,2.818594
4,-3.0,6.591809,-3.476684,1.056816,-3.391863,-0.0270,9.0,-0.474131,1.665136,0.148655,-4.310493,-2.735751,2.567452
5,-4.0,10.106026,-3.747312,0.799578,-6.062053,-0.0064,16.0,-0.962309,1.558399,0.236517,-6.653938,-3.413978,3.178998
6,-2.0,10.021458,-1.858124,0.822762,-3.089820,-0.0284,4.0,-0.851503,1.505785,0.276679,-3.241517,-1.771167,3.165669
7,-2.0,8.199208,-1.979472,0.815589,-2.601852,-0.0024,4.0,-0.799624,1.546463,0.195286,-3.125000,-1.648094,2.863426
8,0.0,7.837506,0.000000,0.891717,0.000000,0.0000,0.0,-0.736865,1.660452,0.185051,0.000000,0.000000,2.799555
9,1.0,7.315203,1.097297,0.860044,1.126943,0.0007,1.0,-0.760186,1.731228,0.150109,1.577720,0.783784,2.704663


In [301]:
px.scatter(x=df_train["date"], y=resid)

In [286]:
pd.concat([
    df_train[["date", "Data da coleta", "Resultado"]].reset_index(drop=True),
    pd.Series(y_pred, name="pred").reset_index(drop=True),
    pd.Series(resid, name="resid").reset_index(drop=True)], axis=1)

,date,Data da coleta,Resultado,pred,resid
0,2019-01-28,2019-01-28,251,90.028310,160.971690
1,2019-01-30,2019-01-28,251,23.371855,227.628145
2,2019-02-07,2019-02-11,335,353.929994,-18.929994
3,2019-02-09,2019-02-11,335,53.575832,281.424168
4,2019-02-22,2019-02-25,298,339.885309,-41.885309
5,2019-03-14,2019-03-18,76,53.705228,22.294772
6,2019-03-16,2019-03-18,76,52.686083,23.313917
7,2019-03-24,2019-03-26,186,42.226298,143.773702
8,2019-03-26,2019-03-26,186,136.064233,49.935767
9,2019-04-10,2019-04-09,26,230.897776,-204.897776


In [287]:
m = clean_gee.plot_img(data_examinada="2021-05-12", config=config)
m

2021-05-12 10:40:45.267000


Map(center=[-30.012175, -51.215679], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

In [288]:
tsne = TSNE(perplexity=5)
df_t = tsne.fit_transform(df_poly_selected)
df_t = pd.concat([pd.DataFrame(df_t), resid.reset_index(drop=True)], axis=1)
px.scatter(df_t, x=0, y=1, color="Resultado")

In [256]:
df_t_date = pd.concat([df_train[["date", "Data da coleta"]].reset_index(drop=True), df_t], axis=1)
df_t_date.loc[(df_t_date[0] > 0) & (df_t_date[1] >0)]

,date,Data da coleta,0,1,Resultado
13,2019-09-22,2019-09-23,30.651655,83.984230,5.991417
19,2019-12-01,2019-12-02,53.918011,108.254982,-5.180981
20,2020-01-05,2020-01-06,66.351273,118.446487,-287.346287
25,2020-08-05,2020-08-06,38.661556,91.247856,233.964358
34,2021-05-12,2021-05-13,55.115898,121.509460,-11926.844998


In [262]:
clean_gee.plot_img(data_examinada="2021-05-12", config=config)


2021-05-12 10:40:45.267000


Map(center=[-30.012175, -51.215679], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…